# 1. Setup ViZDoom

In [1]:
!pip install vizdoom

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!cd github_clone & git clone https://github.com/Farama-Foundation/ViZDoom

fatal: destination path 'ViZDoom' already exists and is not an empty directory.


In [3]:
# ViZDoom for game env
from vizdoom import *
# Random for action sampling
import random
# Time for sleeping
import time
# Identity matrix
import numpy as np

In [8]:
# Setup game
game = DoomGame()
game.load_config('github_clone/ViZDoom/scenarios/basic.cfg')
game.init()

In [5]:
# Define the set of actions within this Doom env
actions = np.identity(3, dtype=np.uint8)

In [ ]:
episodes = 10

for episode in range(episodes):
    # Each episode is a replay of a game
    game.new_episode()
    
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        # Ammo
        info = state.game_variables
        # Take action, skip next 4 frames, and return the result (better for our agent to see the action take place)
        reward = game.make_action(random.choice(actions), 4)
        print('reward: ', reward)
        time.sleep(0.02)
    print('Result: ', game.get_total_reward())
    time.sleep(2)

In [ ]:
game.close()

# 2. Convert to Gym Environment

# 3. View Game State

# 4. Setup Callback

# 5. Train the Model

# 6. Load the Model